# Making a Reddit App for Authorization

1. Make a Reddit account and log in
2. Go to https://www.reddit.com/prefs/apps/
3. Create an App
4. Fill out the create application form 
  1. Choose the "script" option
  2. For our class, a redirect uri of http://soic.indiana.edu will suffice
5. After you've created the app, you'll see a window with your app's settings
  1. Get the client id - it's under your app's name
  2. Get the client secret
  
# Creating PRAW Reddit api object

### The parameters in the variables below are as follows: 

- client_id='PERSONAL_USE_SCRIPT_14_CHARS', \
- client_secret='SECRET_KEY_27_CHARS ', \
- user_agent='YOUR_APP_NAME', \
- username='YOUR_REDDIT_USER_NAME', \
- password='YOUR_REDDIT_LOGIN_PASSWORD')

In [26]:
import praw
import pandas as pd
import datetime as dt

client_id = "WhkpjLo6_5t5zQ" # insert your client ID here
client_secret = "nZhrnOnulzDse-k6AujCKkGPyh4" # client secret here
user_agent = "IU-SMM-2" # a string identifying your app to agents; it is courteous practice to provide your contact info
# username = "psuaggie"

r = praw.Reddit(client_id=client_id, client_secret=client_secret, user_agent=user_agent)

## Getting a User and Their Submissions

In [6]:
gaming = r.subreddit('gaming')

In [6]:
user = r.redditor("bboe") # bboe is a PRAW developer
print(user.name)

bboe


Get the user's 10 most recent submissions.

In [19]:
stored_submissions = []
for submission in user.submissions.new(limit=10):
    stored_submissions.append(submission)

Submission objects have multiple useful properties.

submission = stored_submissions[1]

print("Submission author?")
print(submission.author)
print("How many upvotes?")
print(submission.ups)
print("And downvotes?")
print(submission.downs)
print("Total score (ups minus downs)")
print(submission.score)
print("If it is a text submission, what's the text?")
print(submission.selftext)

## Get a user's comments and scores

stored_comments = []

for comment in user.comments.new(limit=10):
    stored_comments.append(comment)

In [ ]:
for comment in stored_comments:
    print(comment.body)
    print(comment.score)
    print("*"*50)

## Other ways of pulling submissions/comments

Note that we called the method ``new`` to get the user's latest comments. There are methods ``hot``, ``top``, and ``controversial`` that order your items differently. ``new`` does reverse chronological, ``top`` gets items with highest scores, ``controversial`` gets items that have similar levels of upvotes and downvotes, ``hot`` gets items that have gotten upvotes recently.

In [ ]:
# bboe's best submissions

for submission in user.submissions.top(limit=5):
    print(submission.title, submission.score)

# Getting Content from Subreddits

Let's make a Pythonic object representing the subreddit.

In [20]:
subreddit_gaming = r.subreddit("gaming")
subreddit_ps4 = r.subreddit("ps4")
## subreddit_xbox360 = r.subreddit("xbox360")
subreddit_wii = r.subreddit("wii")
subreddit_xboxone = r.subreddit("xboxone")

Get most recent comments and scores.

In [24]:
comments_gaming = []
comments_ps4 = []
## comments_xbox360 = []
comments_wii = []
comments_xboxone = []

for c in subreddit_gaming.comments(limit=10):
    comments_gaming.append((c.body, c.score))
    
for c in subreddit_xboxone.comments(limit=10):
    comments_xboxone.append((c.body, c.score))    

for c in subreddit_ps4.comments(limit=10):
    comments_ps4.append((c.body, c.score))

##for c in subreddit_xbox360.comments(limit=10):
  ##  comments_xbox360.append((c.body, c.score))

for c in subreddit_wii.comments(limit=10):
    comments_wii.append((c.body, c.score))


In [25]:
print(comments_gaming[0:10])
print('*'* 50)

print(comments_ps4[0:10])
print('*'* 50)

##print(comments_xbox360[0:10])
##print('*'* 50)

print(comments_xboxone[0:10])
print('*'* 50)

print(comments_wii[0:10])


[('Dad why my pp moving', 1), ('Dead Space inVR would be the tits! ', 1), ("It's in a great state right now, at launch there were several game-breaking bugs, but those have all been fixed. There are a few visual glitches here and there, but it's about on par with a Skyrim or Fallout in terms of random glitches and non-game breaking bugs.  ", 1), ('I’m dyslexic I can read this fine!', 1), ('What’s the spatula for? Support?', 1), ('I loved it, sold me on SEGA.', 1), ('Cool now get Bloodborne', 1), ('My steam, Xbox, and PS4 names are all Chumpchees3. I play on all three.', 1), ('Sleeping potion?', 1), ("It's pretty much 50/50 right now for a lot of people. Half of the people say most of the bugs are fixed and it's fine. Others, like myself, still have problems with a lot of bugs. The biggest and most annoying bug that I found that actually made me stop playing the game is items from my inventory, my horses inventory, AND my personal storage chest would randomly disappear. People have been

## Get Highest Rated Submissions from Last Week

In [52]:
r_top_submissions = []
d_top_submissions = []

for s in subreddit_republican.top(limit=10, time_filter="week"):
    r_top_submissions.append((s.title, s.score))
    
for s in subreddit_democrat.top(limit=10, time_filter="week"):
    d_top_submissions.append((s.title, s.score))

In [53]:
print(r_top_submissions[0])
print(d_top_submissions[0])

('George H. W. Bush Receives Huge Ovation Before Super Bowl Coin Toss', 308)
('Nordstrom drops Ivanka Trump-branded clothing and shoes', 4208)


# Getting comments on submissions

Getting comments on submissions is a little complicated. With users or entire subreddits, it's simple, because there is a one-to-many correspondence between user/subreddit and the comments it has. However, comments on a submission are organized in a *tree-like structure*; that is, the submission itself may have comments, and those comments may have comments on them, and so on. Because of this, we don't have helpful organizing functions like ``new`` or ``top``. We have to get them all and organize them ourselves. 

First, let's get the most recent submission of a subreddit and try to get the *top-level comments* on that submission. 

In [70]:
newssubreddit = r.subreddit("news")

submissions = []

for submission in newssubreddit.top(limit=5, time_filter="week"):
    submissions.append(submission)
    
s = submissions[0] # Let's work with the first submission

In [71]:
comments = []

for top_level_comment in s.comments:
    comments.append(top_level_comment)

In [79]:
# Text of the 11th comment
comments[0].body

'There are lots of reddit comments in political posts like these with opinions and feelings on the matter. I did some reading up for a bit on what powers the federal court are, how I think they apply, and what implications they have here to help people out. (Shout out to /u/victor_e_bull for some of this info)\n\n**What does the ruling say?**\n\nFirst and foremost the actual ruling can be read [here](http://documents.latimes.com/read-federal-judges-temporary-restraining-order-against-trumps-travel-ban/). Although the case itself deals with only a list of plaintiffs, the judge has issued an order to the defendants (the US government, Trump, and the agencies involved) that apply to not only these plaintiffs but all immigrants.\n\n> It IS HEREBY ORDERED THAT:\n\n>1) Defendants and their officers, agents, employees, attorneys, and all other persons acting in concert or participating with them are, ENJOINED AND RESTRAINED from enforcing Defendant Donald J Trump\'s January 27, 20017 Executiv

Comments can have comments themselves. Here's how extract the children comments of the first comment on the original submission.

In [84]:
replies = []

for reply in comments[0].replies:
    replies.append(reply)

In [85]:
replies[0].body

'The revocation rules you listed are for Consular Offices.  The State Department can revoke for no reason at all.'

PRAW deals with Reddit rate limitations on comments by inserting "MoreComments" objects into the comment tree. For example, at the time of me writing this code, the fourth item in replies is a "MoreComments" object.

In [90]:
replies

[Comment(id='dd9wf5m'),
 Comment(id='dda286y'),
 Comment(id='dda7p6d'),
 <MoreComments count=31, children=['dd9yfy8', 'dda3r4z', 'dda3ia2', '...']>]

We can open up a MoreComments object, but this necessitates sending another request to reddit.

In [93]:
mc = replies[-1]

comments = []
for c in mc.comments():
    comments.append(c)

In [94]:
comments[0].body

'Thank you for your work'

It's important to keep in mind if you work with reddit Comment Forests to calibrate your code to handle "MoreComments" objects gracefully. If you need help writing the code to do this, let me know.